# LECL(대화내용) 기억하기: 메모리 추가

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

In [3]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# MessagesPlaceholder 공간만 잡아준다. 비어져 있는 상태에서, 대화를 쭉 이어나가면 MessagesPlaceholder에 대화기록이 쌓이게 된다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful chatbots"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{input}")
    ]
)2

In [13]:
# 메모리 버퍼를 생성하고, 메세지를 반환 기능을 활성화.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
 return_messages=True, memory_key="chat_history"

In [14]:
# 메모리를 딕셔너리로 변환한다. 여기서는 메모리가 비어져 있으므로, 빈 딕셔너리로 초기화한다.
memory.load_memory_variables({})

{'chat_history': []}

`RunnablePassthrough.assign`을 사용하여 `chat_history`변수에 `memory_load_memory_variables` 함수의 결과를 할당하고(매핑), 이 결과에서 `chat_history`키에 해당하는 값을 추출한다.

In [6]:
# runnable = RunnablePassthrough.assign( # assign은 input과 chat_history에 해당하는 key, value쌍을 반환해주는 것이다.
#     chat_history=RunnableLambda(memory.load_memory_variables) # load_memory_variables 함수를 호출한것과 같다. 즉, 지금까지 쌓아온 함수를 호출을 하는것.
#     # | itemgetter("chat_history") # memory_key와 동일하게 입력한다.
# )

In [7]:
# # input은 사용자의 질문, chat_history는 대화의 기록
# runnable.invoke({"input":"hi"})

{'input': 'hi', 'chain_history': {'chat_history': []}}

In [25]:
runnable = RunnablePassthrough.assign( # assign은 input과 chat_history에 해당하는 key, value쌍을 반환해주는 것이다.
    chat_history=RunnableLambda(memory.load_memory_variables) # load_memory_variables 함수를 호출한것과 같다. 즉, 지금까지 쌓아온 함수를 호출을 하는것.
    | itemgetter("chat_history") # memory_key와 동일하게 입력한다. itemgetter의 역할은 dictionary가 주어졌을때, 해당 value로 끄집어 내주는 역할을 하는것이다.
)


In [26]:
# input은 사용자의 질문, chat_history는 대화의 기록. key, value쌍이 아닌 list만 나오게 된다.
runnable.invoke({"input":"hi"})

{'input': 'hi', 'chat_history': []}

`runnable`에 첫대화를 시작.
- `input`: 사용자의 입력 대화가 전달
- `chat_history`: 대화 기록이 전달

In [27]:
runnable.invoke({"input":"hi"})

{'input': 'hi', 'chat_history': []}

In [28]:
chain = runnable | prompt | model

In [30]:
response = chain.invoke({"input": "만나서 반갑습니다. 제 이름은 재호입니다."})
print(response.content)

안녕하세요, 재호님! 만나서 반갑습니다. 어떻게 도와드릴까요?


In [31]:
memory.load_memory_variables({})

{'chat_history': []}

In [32]:
memory.save_context(
    {"human": "만나서 반갑습니다. 제 이름은 테디입니다"}, {"ai": response.content}
)

# 저장된 대화기록을 출력
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반갑습니다. 제 이름은 테디입니다'),
  AIMessage(content='안녕하세요, 재호님! 만나서 반갑습니다. 어떻게 도와드릴까요?')]}

In [34]:
response = chain.invoke({"input": "제 이름이 무엇이었는지 기억하세요?"})
print(response)

content='네, 당신의 이름은 테디입니다. 어떻게 도와드릴까요?' response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 69, 'total_tokens': 86}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None} id='run-17bdd99b-d1ce-44ac-a10a-7a663e2168c1-0' usage_metadata={'input_tokens': 69, 'output_tokens': 17, 'total_tokens': 86}


### 커스텀 ConversationChain구성 예시

In [7]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import Runnable, RunnablePassthrough, RunnableLambda
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# llm과 prompt, memory만 있으면 된다.

llm = ChatOpenAI(model_name="gpt-4o-mini")

# 대화형 프롬프트를 생성. 이 프롬프트는 시스템 메세지, 이전대화 내역, 그리고 사용자의 입력 포함
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ('human', "{input}")
    ]
)

# 대화 버퍼 메로리를 새성하고, 메세지 반환기능 활성화
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

# runnable = RunnablePassthrough.assign(
#     chat_history=RunnableLambda(memory.load_memory_variables)
#     | itemgetter("chat_history")
# )
# chain = runnable | prompt | llm

In [9]:
class MyConversationChain(Runnable):
    # llm, prompt, memory를 받아서 저장
    def __init__(self, llm, prompt, memory, input_key="input"):
        
        self.prompt = prompt
        self.memory = memory
        self.input_key = input_key
        # 체인을 받아준다.
        self.chain = (
            RunnablePassthrough.assign(
                chat_history=RunnableLambda(self.memory.load_memory_variables)
                | itemgetter(memory.memory_key) # memory_key와 동일하게 입력
            )
            | prompt
            | llm
            | StrOutputParser()
        )
        
    # invoke라는 함수를 통해서, 유저의 질문을 받아서 chain에 invoke하고, 답변을 받은다음 메모리에 저장하는 기능
    def invoke(self, query, config=None, **kwargs):
        answer = self.chain.invoke({self.input_key:query})
        self.memory.save_context(inputs={"human": query}, outputs={'ai': answer})
        return answer

In [10]:
conversation_chain = MyConversationChain(llm, prompt, memory)

In [11]:
conversation_chain.invoke("안녕하세여? 반갑습니다 제 이름은 재호입니다.")

'안녕하세요, 재호님! 반갑습니다. 어떻게 도와드릴까요?'

In [12]:
conversation_chain.invoke("제 이름이 뭐라고요?")

'재호님이라고 하셨습니다! 맞나요?'

In [13]:
conversation_chain.invoke("앞으로는 영어로만 답해주세요")

'Sure, I can respond in English from now on! How can I assist you today?'

In [14]:
conversation_chain.invoke("제 이름을다시한번 말해주세요")

'Your name is Jae-ho.'

In [15]:
conversation_chain.memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='안녕하세여? 반갑습니다 제 이름은 재호입니다.'),
  AIMessage(content='안녕하세요, 재호님! 반갑습니다. 어떻게 도와드릴까요?'),
  HumanMessage(content='제 이름이 뭐라고요?'),
  AIMessage(content='재호님이라고 하셨습니다! 맞나요?'),
  HumanMessage(content='앞으로는 영어로만 답해주세요'),
  AIMessage(content='Sure, I can respond in English from now on! How can I assist you today?'),
  HumanMessage(content='제 이름을다시한번 말해주세요'),
  AIMessage(content='Your name is Jae-ho.')]}

: 

In [68]:
conversation_chain.memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='안녕하세여? 반갑습니다 제 이름은 재호입니다.'),
 AIMessage(content='안녕하세요, 재호님! 반갑습니다. 어떻게 도와드릴까요?'),
 HumanMessage(content='제 이름이 뭐라고요?'),
 AIMessage(content='재호님이라고 하셨습니다. 맞나요?'),
 HumanMessage(content='앞으로는 영어로만 답해주세요'),
 AIMessage(content='Sure, I can respond in English from now on. How can I assist you today?'),
 HumanMessage(content='제 이름을다시한번 말해주세요'),
 AIMessage(content='Your name is Jaeho.')]